In [1]:
# intala o pyspark

!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7cc2f5d5477d5da970c1733a897af36f093b2a111173ec6979ca5d3c6432b992
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [11]:
#inicia a sessão e faz importação

from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.types import *
from pyspark.sql import functions as Func
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()


In [4]:
df1 = spark.createDataFrame([("Pedro",10),("Maria",20),("José",40)])
#show é ação, então tudo o que foi feito anteriormente é executado, lazzy
df1.show()


+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| José| 40|
+-----+---+



In [5]:
#criar df com schema
#from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("Test").getOrCreate()
schema = "Id INT, Nome STRING"
dados = [[1,"Pedro"],[2,"Maria"]]
df2 = spark.createDataFrame(dados, schema)
df2.show()
df2.show(1)

+---+-----+
| Id| Nome|
+---+-----+
|  1|Pedro|
|  2|Maria|
+---+-----+

+---+-----+
| Id| Nome|
+---+-----+
|  1|Pedro|
+---+-----+
only showing top 1 row



In [7]:
#com transformação
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").getOrCreate()

from pyspark.sql.functions import sum
schema2 = "Produtos STRING, Vendas INT"
vendas = [["Caneta",10],["Lápis",20],["Caneta",40]]
df3 = spark.createDataFrame(vendas, schema2)
df3.show(1)



+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
+--------+------+
only showing top 1 row



In [8]:
#agrupamentpo
agrupado = df3.groupBy("Produtos").agg(sum("Vendas"))
agrupado.show()

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [9]:
#selecionar colunas específicas
df3.select("Produtos").show()
df3.select("produtos","Vendas").show()

+--------+
|Produtos|
+--------+
|  Caneta|
|   Lápis|
|  Caneta|
+--------+

+--------+------+
|produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [12]:
#expressões e select

df3.select("Produtos", "Vendas", expr("(Vendas * 0.2) + Vendas").alias("Vendas_Com_Acrescimo")).show()

+--------+------+--------------------+
|Produtos|Vendas|Vendas_Com_Acrescimo|
+--------+------+--------------------+
|  Caneta|    10|                12.0|
|   Lápis|    20|                24.0|
|  Caneta|    40|                48.0|
+--------+------+--------------------+



In [13]:
#para ver o schema
# Supondo que df3 seja o seu DataFrame e já esteja definido
df3.printSchema()
df3.schema

root
 |-- Produtos: string (nullable = true)
 |-- Vendas: integer (nullable = true)



StructType([StructField('Produtos', StringType(), True), StructField('Vendas', IntegerType(), True)])

In [15]:
#importar dados definindo schema
#vamos deixar a data como string de propósito

arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
#o caminho pode mudar, download é a pasta que você baixou com dados de exemplo

despachantes = spark.read.csv("despachantes.csv", header=False, schema=arqschema)
despachantes.show()



+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [16]:
#outro exemplo, inferindo schema, usando load e informado tipo
#a leitura do arquivo vai variar para cada maquina
desp_autoschema = spark.read.load("despachantes.csv",
                     format="csv", sep=",", inferSchema=True, header=False)
desp_autoschema.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [17]:
#comparando os schemas
desp_autoschema.schema
despachantes.schema


desp_autoschema.printSchema()
#despachantes.printSchema()despachantes

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: date (nullable = true)



In [18]:
#condição lógica com where

despachantes.select("id", "nome", "vendas").where(Func.col("vendas")> 20).show()

#& para and, | para or, e ~ para not
despachantes.select("id","nome","vendas").where((Func.col("vendas")> 20) & (Func.col("vendas") < 40)).show()


+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [19]:
#renomear coluna
novodf = despachantes.withColumnRenamed("nome","nomes")
novodf.columns

['id', 'nomes', 'status', 'cidade', 'vendas', 'data']

In [20]:
#coluna data está como string, vamos transformar em texto
despachantes2 = despachantes.withColumn("data2", to_timestamp(Func.col("data"),"yyyy-MM-dd"))
despachantes2.schema

StructType([StructField('id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True), StructField('data2', TimestampType(), True)])

In [21]:
#despachantes2.select(year("data")).show()
despachantes2.select(year("data")).distinct().show()
despachantes2.select("nome",year("data")).orderBy("nome").show()
despachantes2.select("data").groupBy(year("data")).count().show()

#despachantes2.select("data").groupBy(year("data")).count().show()


+----------+
|year(data)|
+----------+
|      2018|
|      2019|
|      2020|
+----------+

+-------------------+----------+
|               nome|year(data)|
+-------------------+----------+
|   Carminda Pestana|      2020|
|    Deolinda Vilela|      2020|
|   Emídio Dornelles|      2020|
|Felisbela Dornelles|      2020|
|     Graça Ornellas|      2020|
|   Matilde Rebouças|      2019|
|    Noêmia   Orriça|      2019|
|      Roque Vásquez|      2020|
|      Uriel Queiroz|      2018|
|   Viviana Sequeira|      2020|
+-------------------+----------+

+----------+-----+
|year(data)|count|
+----------+-----+
|      2018|    1|
|      2019|    2|
|      2020|    7|
+----------+-----+



In [22]:
#transformações
despachantes.take(3)
despachantes.collect()
despachantes.count()
despachantes.orderBy("vendas").show()
despachantes.orderBy(Func.col("vendas").desc()).show()
despachantes.orderBy(Func.col("vendas").desc()).show()
despachantes.orderBy(Func.col("cidade").desc(),Func.col("vendas").desc()).show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
+---+-------------------+------+-------------+------+----------+

+---+-------------------+------+-------------+------+----------+
| id|               nome

In [23]:
# agregação e filtros
despachantes.groupBy("cidade").agg(sum("vendas")).show()

despachantes.groupBy("cidade").agg(sum("vendas")).orderBy(Func.col("sum(vendas)").desc()).show()

despachantes.filter(Func.col("nome") == "Deolinda Vilela").show()

despachantes.groupBy("cidade").agg(sum("vendas")).where(Func.col("sum(vendas)")> 70 ).orderBy(Func.col("sum(vendas)").desc()).show()



+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|  Santa Maria|         68|
|Novo Hamburgo|         34|
| Porto Alegre|        223|
+-------------+-----------+

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+

+---+---------------+------+-------------+------+----------+
| id|           nome|status|       cidade|vendas|      data|
+---+---------------+------+-------------+------+----------+
|  2|Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
+---+---------------+------+-------------+------+----------+

+------------+-----------+
|      cidade|sum(vendas)|
+------------+-----------+
|Porto Alegre|        223|
+------------+-----------+



In [27]:
#exportando arquivos

despachantes.write.mode("overwrite").format("json").save("despachantes")
despachantes.write.mode("overwrite").format("orc").save("dfimportorc")
despachantes.write.mode("overwrite").format("csv").save("dfimportcsv")
despachantes.write.mode("overwrite").format("parquet").save("dfimportparquet")

